In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile

from loguru import logger

import numpy as np
import pytorch_lightning as pl
import torch
import datamol as dm
import pandas as pd

import goli

Using backend: pytorch


In [2]:
GOLI_DATASETS = ["ZINC-micro.zip", "ZINC-bench-gnn.zip", "htsfp-t20000_full.csv.gz"]

In [41]:
name = "ZINC-micro.zip"
extract_zip = True
output_path = "/home/hadim/goli-datasets"
progress = False

args = {}
args["name"] = "ZINC-micro"
args["output_path"] = "/home/hadim/goli-datasets"
args["extract_zip"] = True
args["progress"] = False

data_path = goli.data.utils.download_goli_dataset(**args)

In [42]:
fpath = goli.utils.fs.join(data_path, "ZINC-micro.csv")

In [43]:
df = pd.read_csv(fpath)
assert df.shape == (1000, 4)
assert df.columns == ["SMILES", "SA", "logp", "score"]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [44]:
df.columns

Index(['SMILES', 'SA', 'logp', 'score'], dtype='object')